<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/XVPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import urllib.parse
from google.colab import files
from datetime import datetime

def descarregar_xvpca(any_seleccionat=None, municipi=None, contaminant=None):
    """
    Descarrega dades de la XVPCA filtrant per any, municipi i/o contaminant.
    Endpoint: https://analisi.transparenciacatalunya.cat/resource/tasf-thgu.json
    """
    base_url = "https://analisi.transparenciacatalunya.cat/resource/tasf-thgu.json"
    limit = 50000
    offset = 0
    tots_els_dades = []

    # Construir condicions SoQL
    condicions = []

    # Filtre per Municipi (Normalitzem text per evitar errors majúscules/minúscules si calgués,
    # però l'API requereix coincidència exacta, així que assumim input correcte)
    if municipi:
        condicions.append(f"municipi='{municipi}'")

    # Filtre per Contaminant (ex: NO2, PM10, O3, H2S...)
    if contaminant:
        condicions.append(f"contaminant='{contaminant}'")

    # Filtre per Data (Any)
    if any_seleccionat:
        data_inici = f"{any_seleccionat}-01-01T00:00:00"
        data_fi = f"{any_seleccionat+1}-01-01T00:00:00"
        condicions.append(f"data >= '{data_inici}' AND data < '{data_fi}'")

    where_clause = " AND ".join(condicions) if condicions else "1=1"

    print("🚀 Iniciant descàrrega de dades XVPCA (Aire)...")
    print(f"   Filtres -> Any: {any_seleccionat}, Municipi: {municipi}, Contaminant: {contaminant}")

    while True:
        # Ordenem per data per tenir una descàrrega seqüencial lògica
        query = f"?$where={where_clause}&$limit={limit}&$offset={offset}&$order=data ASC"
        url_neta = base_url + urllib.parse.quote(query, safe='?=&-:T')

        try:
            # Llegim el JSON directament a Pandas
            df_bloc = pd.read_json(url_neta)

            if df_bloc.empty:
                break

            tots_els_dades.append(df_bloc)
            offset += limit

            # Mostrem progrés
            ultima_data = df_bloc['data'].iloc[-1]
            print(f"📥 {offset} registres processats... Data actual: {str(ultima_data)[:10]}")

            # Petita pausa per no saturar l'API
            time.sleep(0.3)

        except Exception as e:
            print(f"⚠️ Error durant la descàrrega: {e}")
            break

    if tots_els_dades:
        df_final = pd.concat(tots_els_dades, ignore_index=True)

        # Convertim columnes de data
        if 'data' in df_final.columns:
            df_final['data'] = pd.to_datetime(df_final['data'])

        print(f"\n✅ ÈXIT: {len(df_final)} registres descarregats.")
        if not df_final.empty and 'data' in df_final.columns:
            print(f"📅 Període: {df_final['data'].min().date()} fins a {df_final['data'].max().date()}")

        # Generació del nom del fitxer dinàmic
        nom_fitxer = f"XVPCA_Aire"
        if municipi: nom_fitxer += f"_{municipi.replace(' ', '_')}"
        if contaminant: nom_fitxer += f"_{contaminant}"
        if any_seleccionat: nom_fitxer += f"_{any_seleccionat}"
        nom_fitxer += ".csv"

        df_final.to_csv(nom_fitxer, index=False)
        files.download(nom_fitxer)

        return df_final
    else:
        print("❌ No s'han trobat dades amb aquests filtres.")
        return None

# --- INTERACCIÓ AMB L'USUARI ---

print("--- DESCARREGADOR DADES AIRE CATALUNYA (XVPCA) ---")
print("Exemples de contaminants: NO2, PM10, O3, CO, SO2")
print("Exemples de municipis: Barcelona, Girona, Badalona, Manlleu")
print("----------------------------------------------------")

any_usuari = input("Introdueix l'any (deixa buit per tots): ").strip()
municipi_usuari = input("Introdueix Municipi (deixa buit per tots): ").strip()
contaminant_usuari = input("Introdueix Contaminant (deixa buit per tots): ").strip().upper() # Forcem majúscules al contaminant

# Processar inputs
any_usuari = int(any_usuari) if any_usuari.isdigit() else None
municipi_usuari = municipi_usuari if municipi_usuari else None
contaminant_usuari = contaminant_usuari if contaminant_usuari else None

# Executar funció
df_resultat = descarregar_xvpca(any_usuari, municipi_usuari, contaminant_usuari)

--- DESCARREGADOR DADES AIRE CATALUNYA (XVPCA) ---
Exemples de contaminants: NO2, PM10, O3, CO, SO2
Exemples de municipis: Barcelona, Girona, Badalona, Manlleu
----------------------------------------------------
Introdueix l'any (deixa buit per tots): 2025
Introdueix Municipi (deixa buit per tots): Martorell
Introdueix Contaminant (deixa buit per tots): 
🚀 Iniciant descàrrega de dades XVPCA (Aire)...
   Filtres -> Any: 2025, Municipi: Martorell, Contaminant: None
📥 50000 registres processats... Data actual: 2025-12-31

✅ ÈXIT: 1896 registres descarregats.
📅 Període: 2025-01-01 fins a 2025-12-31


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>